In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_excel("/content/National level.xlsx",index_col=0)
data.tail()

,NewCase,Newrecov,Newtest,Newdeath
2021-08-26,7357,9522,37875,97
2021-08-27,6863,6188,37167,92
2021-08-28,6175,7163,35131,93
2021-08-29,3841,6815,25236,76
2021-08-30,2676,7360,14838,103


In [3]:
NewCase=data["NewCase"].values

In [4]:
def split_data(array,input_step,horizon):
    X,Y=[],[]
    for i in range(len(array)):
        idx_last_y=i+input_step+horizon
        if idx_last_y <= len(array):
            Y.append(array[idx_last_y-horizon:idx_last_y])
            X.append(array[i:i+input_step])
        else :
            break
    return np.array(X),np.array(Y)

In [5]:
#Prepare data for LSTM
lag=10
horizon=7
X,Y=split_data(NewCase,lag,horizon)

In [6]:
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
model = keras.Sequential([
    layers.LSTM(200, activation='relu',return_sequences=True, input_shape=(10, 1)),
    layers.LSTM(200,activation='relu'),
    layers.Dense(7),
])
model.compile(optimizer='adam', loss='mae')

In [9]:
history = model.fit(
    X, Y,
    epochs=150,
    verbose=0,
)

In [10]:
var=data.tail(10)['NewCase'].values
var=np.array(var)
var=var.astype(float)
var=var.reshape(1,10,1)
Y_hat=model.predict(var)
Y_hat

array([[6331.47  , 6537.4336, 6146.505 , 5857.841 , 5572.5864, 5380.3413,
        6121.274 ]], dtype=float32)

In [11]:
#For deaths
lag=10
horizon=7
n_features = 1
Newdeath=data["Newdeath"].values
X,Y=split_data(Newdeath,lag,horizon)
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [12]:
model = keras.Sequential([
    layers.LSTM(50, activation='relu',return_sequences=True, input_shape=(10, 1)),
    layers.LSTM(50,activation='relu'),
    layers.Dense(7),
])
model.compile(optimizer='adam', loss='mae')

In [13]:
history = model.fit(
    X, Y,
    epochs=50,
    verbose=0,
)

In [14]:
var=data.tail(10)['Newdeath'].values
var=np.array(var)
var=var.astype(float)
var=var.reshape(1,10,1)
Y_hat=model.predict(var)
Y_hat

array([[71.46516, 77.29327, 93.75049, 86.33832, 71.20515, 68.16835,
        67.11404]], dtype=float32)